In [2]:
import numpy
import os
import glob
import matplotlib.pyplot as plt
from skimage import transform as tf
import math 
%matplotlib inline

In [5]:
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
dataLocation = ['/Users/gattia/Data/mri/ski10Dataset/']
trainingImages = 'train_images.npy'
trainingLabels = 'train_labels.npy'

img_rows = 448 
img_cols = 352
numberClasses = 5

smooth = 1.

In [4]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    classes = y_true.shape[1]
    dices = []
    for label in range(len(classes)):
        singleClass_true = y_true_f[:,label]
        singleClass_pred = y_pred_f[:,label]
        dices[label] = K.sum(singleClass_pred[singleClass_true==0])*2.0 / (K.sum(singleClass_pred) + K.sum(singleClass_true))
    return (K.mean(dices))


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [6]:
def get_net():
    inputs = Input((1,img_rows, img_cols))
    conv1 = Convolution2D(8,5,5, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(8,5,5, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 = Convolution2D(8,5,5, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(8,5,5, activation='relu', border_mode='same')(conv2)
    up2 = UpSampling2D(size=(2,2))(conv2)
    
    conv3 = Convolution2D(8,5,5, activation='relu', border_mode='same')(up2)
    conv3 = Convolution2D(8,5,5, activation='relu', border_mode='same')(conv3)
    
    conv4 = Convolution2D(5,1,1, activation='sigmoid', border_mode='same')(conv3)
    
    model = Model(input=inputs, output=conv4)
    model.compile(optimizer =Adam(lr=1e-5), loss=dice_coef_loss, metrics =[dice_coef])
    
    return model  

In [ ]:
def train_and_predict():
    